In [22]:
import numpy as np
import matplotlib as plt
import pandas as pd


In [24]:
path = "iris.txt"
df = pd.read_csv(path)
print(df.head())
# Column labels.
headers = ["sepal length", "sepal width", "petal length", "petal width" , "label"]

df.columns = headers

# Converting labels to numeric.
label_map = {"Iris-setosa":0, "Iris-versicolor":1, "Iris-virginica":2}
df['label'] = df['label'].map(label_map)

train = df.sample(frac=0.8,random_state=200)
print(train.shape)

test = df.drop(train.index)
training_dataset = train.values
print(train.values)
testing_dataset = test.values


   5.1  3.5  1.4  0.2  Iris-setosa
0  4.9  3.0  1.4  0.2  Iris-setosa
1  4.7  3.2  1.3  0.2  Iris-setosa
2  4.6  3.1  1.5  0.2  Iris-setosa
3  5.0  3.6  1.4  0.2  Iris-setosa
4  5.4  3.9  1.7  0.4  Iris-setosa
(119, 5)
[[6.  3.4 4.5 1.6 1. ]
 [6.3 2.7 4.9 1.8 2. ]
 [4.7 3.2 1.6 0.2 0. ]
 [5.  3.  1.6 0.2 0. ]
 [6.8 2.8 4.8 1.4 1. ]
 [6.5 3.2 5.1 2.  2. ]
 [5.8 2.7 3.9 1.2 1. ]
 [5.7 2.8 4.1 1.3 1. ]
 [5.5 2.4 3.7 1.  1. ]
 [5.8 2.7 5.1 1.9 2. ]
 [7.2 3.2 6.  1.8 2. ]
 [4.6 3.1 1.5 0.2 0. ]
 [5.  3.2 1.2 0.2 0. ]
 [4.8 3.  1.4 0.3 0. ]
 [7.2 3.  5.8 1.6 2. ]
 [5.8 4.  1.2 0.2 0. ]
 [5.6 2.7 4.2 1.3 1. ]
 [4.4 3.2 1.3 0.2 0. ]
 [5.6 2.9 3.6 1.3 1. ]
 [6.  3.  4.8 1.8 2. ]
 [5.4 3.4 1.7 0.2 0. ]
 [5.9 3.  4.2 1.5 1. ]
 [6.1 3.  4.9 1.8 2. ]
 [4.8 3.  1.4 0.1 0. ]
 [5.3 3.7 1.5 0.2 0. ]
 [5.1 3.8 1.5 0.3 0. ]
 [6.2 2.9 4.3 1.3 1. ]
 [6.7 2.5 5.8 1.8 2. ]
 [6.9 3.1 5.1 2.3 2. ]
 [6.5 2.8 4.6 1.5 1. ]
 [5.  3.5 1.3 0.3 0. ]
 [6.3 3.3 6.  2.5 2. ]
 [5.  2.3 3.3 1.  1. ]
 [5.4 3.9 1.7 0.4 0. ]

In [3]:
classes = 3
X = training_dataset[:, :-1].T
Y_orig = (training_dataset[:, -1].reshape(1,X.shape[1])).astype(int)
X_test = testing_dataset[:, :-1].T
Y_test_orig = (testing_dataset[:, -1].reshape(1,X_test.shape[1])).astype(int)

m = X.shape[1]

#Modifying Y for multi class Classification
Y = np.zeros((m, classes))
Y[np.arange(m), Y_orig] = 1
Y = Y.T

Y_test = np.zeros((X_test.shape[1], classes))
Y_test[np.arange(X_test.shape[1]), Y_test_orig] = 1
Y_test = Y_test.T


print ('The shape of X is: ' + str(X.shape))
print ('The shape of Y is: ' + str(Y.shape))
print ('I have m = %d training examples!' % (m))


The shape of X is: (4, 119)
The shape of Y is: (3, 119)
I have m = 119 training examples!


In [4]:
def layer_sizes(X, Y):
    n_x = X.shape[0] # size of input layer
    n_h = 4
    n_y = Y.shape[0] # size of output layer
    
    return (n_x, n_h, n_y)

In [5]:
def initialize_parameters(n_x, n_h, n_y):
    
    W1 = np.random.randn(n_h,n_x) * 0.01
    b1 = np.zeros((n_h,1)) 
    W2 = np.random.randn(n_y,n_h) * 0.01
    b2 = np.zeros((n_y,1)) 
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [6]:
def sigmoid(X):
    
    return 1.0 / (1.0 + np.exp(-1.0 * X))


In [7]:
def softmax(X):
    exps = np.exp(X)
    return exps / exps.sum(axis=0, keepdims=True)

In [8]:
def forward_propagation(X, parameters):
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    
    Z1 = np.dot(W1,X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2,A1) + b2
    A2 = softmax(Z2)
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [9]:
def compute_cost(A2, Y, parameters):
    
    m = Y.shape[1]
    
    logprobs = np.multiply(np.log(A2),-Y)
    #column wise
    cost = np.sum(logprobs,axis=0,keepdims=True)
    #over all training examples
    cost = np.sum(cost)/m
    cost = np.squeeze(cost)
    return cost

In [10]:
def backward_propagation(parameters, cache, X, Y):
    
    m = X.shape[1]
    
    W1 = parameters['W1']
    W2 = parameters['W2']
    
    A1 = cache['A1']
    A2 = cache['A2']
    
    dZ2 = A2 - Y
    dW2 = (1/m) * np.dot(dZ2,A1.T)
    db2 = (1/m) * np.sum(dZ2,axis = 1, keepdims = True)
    dZ1 = np.multiply(np.dot(W2.T,dZ2),(1 - np.power(A1,2)))
    dW1 = (1/m) * np.dot(dZ1,X.T)
    db1 = (1/m) * np.sum(dZ1,axis=1,keepdims=True)
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [11]:
def update_parameters(parameters, grads, learning_rate = 0.5):
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    
    # Update rule for each parameter
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [12]:
def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=False):
    
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    for i in range(0, num_iterations):
        
        # Forward propagation
        A2, cache = forward_propagation(X, parameters)
   
        # Cost function
        cost = compute_cost(A2, Y, parameters)
 
        # Backpropagation
        grads = backward_propagation(parameters, cache, X, Y)
 
        # Gradient descent parameter update
        parameters = update_parameters(parameters, grads)
        
        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

In [13]:
def predict(parameters, X):
    
    A2, cache = forward_propagation(X, parameters)
    predictions = (A2 == A2.max(axis=0)[None,:]).astype(int)
    
    return predictions

In [14]:
# Build a model with a n_h-dimensional hidden layer
parameters = nn_model(X, Y, n_h = 4, num_iterations = 10000, print_cost=True)

Cost after iteration 0: 1.098718
Cost after iteration 1000: 0.305520
Cost after iteration 2000: 0.089938
Cost after iteration 3000: 0.082545
Cost after iteration 4000: 0.064330
Cost after iteration 5000: 0.063106
Cost after iteration 6000: 0.082633
Cost after iteration 7000: 0.058052
Cost after iteration 8000: 0.049267
Cost after iteration 9000: 0.042715


In [15]:
# Print accuracy
predictions = predict(parameters, X_test)
correct_classified = np.sum(np.multiply(Y_test,predictions),axis=0)

print ('Accuracy: %d' % float( np.sum(correct_classified)/float(Y_test.shape[1])*100) + '%')

Accuracy: 100%
